# Compare mock communities from the NovaSeq 6000 run 

These mock communities have been used as positive controls within the Alm Microbiome project (see analysis here: https://github.com/LenaFloerl/Alm-Microbiome.git).


In [1]:
wd = '/home/lfloerl/cloud/lfloerl/Alm-Microbiome/artifacts'
%cd $wd 

/home/lfloerl/cloud/lfloerl/Alm-Microbiome/artifacts


In [143]:
import qiime2 as q2
import qiime2.plugins.taxa.actions as taxa_actions
from qiime2 import Visualization, Metadata, Artifact
from qiime2.plugins.taxa.methods import collapse
from qiime2.plugins.feature_table.methods import (merge_seqs, merge, filter_seqs, filter_samples, filter_features) 
import qiime2.plugins.feature_classifier.actions as feature_classifier_actions
import qiime2.plugins.metadata.actions as metadata_actions
from qiime2.plugins import taxa as q2t

from biom import load_table
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches

%matplotlib inline

## Prepare metadata

In [110]:
ctrl_md = pd.read_csv('/home/lfloerl/Compare-Mock-Communities/Metadata/NovaSeq6000-controls-metadata.csv')
run_md = pd.read_csv('/home/lfloerl/Compare-Mock-Communities/Metadata/Order_35731_sample.csv')

In [116]:
## ITS

# Extract the part from the 'Name' in run_md
run_md['ITS_ID'] = run_md['Name'].apply(lambda x: '-'.join(x.split('-')[-3:]))

# merge
fun_md = pd.merge(ctrl_md, run_md, left_on=ctrl_md['unique_ITS_ID'], right_on=run_md['ITS_ID'], how='left')
fun_md.set_index('Name', inplace=True)
fun_md.index.names = ['ID']
fun_md.rename(columns={'Sample_ID':'Sample_Name'}, inplace=True)
fun_md = fun_md[['Sample_Name', 'Ctrl', 'DNA_extraction_plate', 'Fungal Conc']]
fun_md['Fungal Conc'] = pd.to_numeric(fun_md['Fungal Conc'], errors='coerce')
# Replace NaN with 0 (optional, depends on how you want to handle NaNs)
fun_md['Fungal Conc'].fillna(0.01, inplace=True)
# Replace negative values with 0
fun_md['Fungal Conc'] = fun_md['Fungal Conc'].apply(lambda x: max(x, 0.01))

fun_md.to_csv('/home/lfloerl/Compare-Mock-Communities/Metadata/NovaSeq6000_fun_metadata.tsv', sep='\t')

In [130]:
fun_md.head()

,Sample_Name,Ctrl,DNA_extraction_plate,Fungal Conc
ID,,,,
357315_058-LP2-ITS-0058,DNA24_NegCtrl_1,neg,DNA24,0.019680
357315_088-LP2-ITS-0088,DNA24_NegCtrl_2,neg,DNA24,2.674231
357315_036-LP2-ITS-0036,DNA24_PosCtrl_1,pos,DNA24,4.563005
357315_094-LP2-ITS-0094,DNA24_PosCtrl_2,pos,DNA24,10.499125
357315_096-LP2-ITS-0096,DNA24_PosCtrl_3,pos,DNA24,12.921753


In [117]:
## 16S

# Extract the part from the 'Name' in run_md
run_md['16S_ID'] = run_md['Name'].apply(lambda x: '-'.join(x.split('-')[-3:]))

# merge
bac_md = pd.merge(ctrl_md, run_md, left_on=ctrl_md['unique_16S_ID'], right_on=run_md['16S_ID'], how='left')
bac_md.set_index('Name', inplace=True)
bac_md.index.names = ['ID']
bac_md.rename(columns={'Sample_ID':'Sample_Name'}, inplace=True)
bac_md = bac_md[['Sample_Name', 'Ctrl', 'DNA_extraction_plate', 'Bacterial Conc']]
# Convert the 'Bacterial Conc' column to numeric, forcing errors to NaN
bac_md['Bacterial Conc'] = pd.to_numeric(bac_md['Bacterial Conc'], errors='coerce')
# Replace NaN with 0 (optional, depends on how you want to handle NaNs)
bac_md['Bacterial Conc'].fillna(0.01, inplace=True)
# Replace negative values with 0
bac_md['Bacterial Conc'] = bac_md['Bacterial Conc'].apply(lambda x: max(x, 0.01))

bac_md.to_csv('/home/lfloerl/Compare-Mock-Communities/Metadata/NovaSeq6000_bac_metadata.tsv', sep='\t')

# ITS

In [155]:
wd = '/home/lfloerl/cloud/lfloerl/Alm-Microbiome/artifacts/ITS'
%cd $wd 

/home/lfloerl/cloud/lfloerl/Alm-Microbiome/artifacts/ITS


In [157]:
rep_seqs = q2.Artifact.load('dada2-3/dada-rep-seqs.qza')
md = Metadata.load('/home/lfloerl/Compare-Mock-Communities/Metadata/NovaSeq6000_fun_metadata.tsv')

### Classify

In [ ]:
fn = '/home/lfloerl/public/Data/Databases/QIIME2/UNITE/sh_qiime_release_10.05.2021/unite-ver8_99-classifier-10.05.2021.qza'
classifier = Artifact.load(fn)

taxonomy, = feature_classifier_actions.classify_sklearn(
    classifier=classifier,
    reads=rep_seqs,
    n_jobs= 10)

taxonomy_as_md_md = taxonomy.view(Metadata)
taxonomy_viz, = metadata_actions.tabulate(
    input=taxonomy_as_md_md)

taxonomy.save('taxonomy.qza')
taxonomy_viz.save('taxonomy.qzv')

### Decontam 

In [ ]:
%%bash

#mkdir Controls

# first we need to subset the table to only contain our controls (decontam cannot handle samples missing in the metadata)
qiime feature-table filter-samples \
  --i-table dada2-3/dada-table.qza \
  --m-metadata-file /home/lfloerl/Compare-Mock-Communities/Metadata/NovaSeq6000_fun_metadata.tsv \
  --o-filtered-table Controls/ctrl_md_filtered_table.qza

qiime feature-table summarize --i-table  Controls/ctrl_md_filtered_table.qza --o-visualization  Controls/ctrl_md_filtered_table.qzv

# DECONTAM 
qiime quality-control decontam-identify \
    --i-table Controls/ctrl_md_filtered_table.qza \
    --m-metadata-file /home/lfloerl/Compare-Mock-Communities/Metadata/NovaSeq6000_fun_metadata.tsv \
    --p-method 'combined' \
    --p-freq-concentration-column 'Fungal Conc' \
    --p-prev-control-column "Ctrl" \
    --p-prev-control-indicator "neg" \
    --o-decontam-scores Controls/decontam_scores.qza \

qiime quality-control decontam-score-viz \
    --i-decontam-scores Controls/decontam_scores.qza \
    --i-table Controls/ctrl_md_filtered_table.qza \
    --o-visualization Controls/decontam_histogram.qzv

# actually remove the contaminations 
qiime quality-control decontam-remove \
    --i-decontam-scores Controls/decontam_scores.qza \
    --i-table Controls/ctrl_md_filtered_table.qza \
    --o-filtered-table Controls/decontam-table.qza \

qiime feature-table summarize --i-table Controls/decontam-table.qza --o-visualization Controls/decontam-table.qzv

### Filter non-target seqs 

In [123]:
# read in the whole list of taxa and lifestyles 
fungi_df = pd.read_csv('/home/lfloerl/RxCS/Metadata/UNITE-FungalTraits.csv')

# these lifestyles are macrofungi! 
fruting_body_forming = ['agaricoid',
 'apothecium_(hymenium on surface)',
 'clathroid',
 'clavarioid',
 'corticioid',
 'cyphelloid',
 'gasteroid',
 'gasteroid-hypogeous',
 'hysterothecium_(opening_slith-like)',
 'mazaedium_(pushpin-like)',
 'phalloid',
 'polyporoid',
 'tremelloid']

# filter accordingly 
matching_taxons = fungi_df[fungi_df['Fruitbody_type_template'].isin(fruting_body_forming)]['Taxon'].tolist()

# we export the taxonomy 
!qiime tools export --input-path taxonomy.qza --output-path taxonomy 

# and import as df to get all Taxons we actually identified
taxonomy_df = pd.read_csv('taxonomy/taxonomy.tsv', sep='\t')

filtered_matching_taxons = [taxon for taxon in matching_taxons if taxon in taxonomy_df['Taxon'].tolist()]

# actual taxons we need to filter out of our FeatureTable 
len(filtered_matching_taxons)

Exported taxonomy.qza as TSVTaxonomyDirectoryFormat to directory taxonomy


969

In [124]:
# Set to hold unique genera
unique_entries = set()

# List to hold entries without 'f__'
entries_without_f = []

# Process each string in the list
for entry in filtered_matching_taxons:
    # Find the start and end of the substring to extract
    start_index = entry.find('f__')
    
    if start_index != -1:
        start_index += len('f__')
        end_index = entry.find(';g__')
        
        if end_index != -1 and end_index > start_index:
            # Extract the substring between 'f__' and ';g'
            substring = entry[start_index:end_index]
            # Add the substring to the set of unique entries
            unique_entries.add(substring)
    else:
        # If 'f__' is not found, add the entry to the list of entries without 'f__'
        entries_without_f.append(entry)

# Convert the set of unique entries to a list
unique_list = list(unique_entries)

# Output the results
print("Unique genera:")
print(unique_list)
print('\n Lenght: ')
print(len(unique_list))
print("\n Just checking - any entries without?")
print(entries_without_f)

Unique genera:
['Auriculariaceae', 'Phallaceae', 'Cyphellaceae', 'Cystostereaceae', 'Russulales_fam_Incertae_sedis', 'Hygrophoropsidaceae', 'Vuilleminiaceae', 'Stephanosporaceae', 'Fomitopsidaceae', 'Amylocorticiaceae', 'Stereopsidaceae', 'Meruliaceae', 'Entolomataceae', 'Radulomycetaceae', 'Stereaceae', 'Corticiaceae', 'Russulaceae', 'Jaapiaceae', 'Ganodermataceae', 'Crepidotaceae', 'Coniophoraceae', 'Phanerochaetaceae', 'Ceratobasidiaceae', 'Sclerodermataceae', 'Boletales_fam_Incertae_sedis', 'Schizoporaceae', 'Hymenochaetaceae', 'Gomphaceae', 'Strophariaceae', 'Meripilaceae', 'Porotheleaceae', 'Schizophyllaceae', 'Bankeraceae', 'Byssocorticiaceae', 'Hymenogastraceae', 'Steccherinaceae', 'Atheliaceae', 'Auriscalpiaceae', 'Omphalotaceae', 'Bondarzewiaceae', 'Boletaceae', 'Tricholomataceae', 'Bolbitiaceae', 'Gloeophyllaceae', 'Cortinariaceae', 'Hyaloriaceae', 'Hyphodermataceae', 'Hygrophoraceae', 'Panaceae', 'Clavariaceae', 'Botryobasidiaceae', 'Pleurotaceae', 'Lycoperdaceae', 'Physala

In [126]:
# we use these to filter our FeatureTable 
exclude_fungi_list = ','.join(unique_list)
exclude_fungi_list

'Auriculariaceae,Phallaceae,Cyphellaceae,Cystostereaceae,Russulales_fam_Incertae_sedis,Hygrophoropsidaceae,Vuilleminiaceae,Stephanosporaceae,Fomitopsidaceae,Amylocorticiaceae,Stereopsidaceae,Meruliaceae,Entolomataceae,Radulomycetaceae,Stereaceae,Corticiaceae,Russulaceae,Jaapiaceae,Ganodermataceae,Crepidotaceae,Coniophoraceae,Phanerochaetaceae,Ceratobasidiaceae,Sclerodermataceae,Boletales_fam_Incertae_sedis,Schizoporaceae,Hymenochaetaceae,Gomphaceae,Strophariaceae,Meripilaceae,Porotheleaceae,Schizophyllaceae,Bankeraceae,Byssocorticiaceae,Hymenogastraceae,Steccherinaceae,Atheliaceae,Auriscalpiaceae,Omphalotaceae,Bondarzewiaceae,Boletaceae,Tricholomataceae,Bolbitiaceae,Gloeophyllaceae,Cortinariaceae,Hyaloriaceae,Hyphodermataceae,Hygrophoraceae,Panaceae,Clavariaceae,Botryobasidiaceae,Pleurotaceae,Lycoperdaceae,Physalacriaceae,Lachnocladiaceae,Paxillaceae,Typhulaceae,Hymenochaetales_fam_Incertae_sedis,Ischnodermataceae,Irpicaceae,Polyporales_fam_Incertae_sedis,Polyporaceae,Thelephoraceae,Ni

In [156]:
%%bash 

filter='Auriculariaceae,Phallaceae,Cyphellaceae,Cystostereaceae,Russulales_fam_Incertae_sedis,Hygrophoropsidaceae,Vuilleminiaceae,Stephanosporaceae,Fomitopsidaceae,Amylocorticiaceae,Stereopsidaceae,Meruliaceae,Entolomataceae,Radulomycetaceae,Stereaceae,Corticiaceae,Russulaceae,Jaapiaceae,Ganodermataceae,Crepidotaceae,Coniophoraceae,Phanerochaetaceae,Ceratobasidiaceae,Sclerodermataceae,Boletales_fam_Incertae_sedis,Schizoporaceae,Hymenochaetaceae,Gomphaceae,Strophariaceae,Meripilaceae,Porotheleaceae,Schizophyllaceae,Bankeraceae,Byssocorticiaceae,Hymenogastraceae,Steccherinaceae,Atheliaceae,Auriscalpiaceae,Omphalotaceae,Bondarzewiaceae,Boletaceae,Tricholomataceae,Bolbitiaceae,Gloeophyllaceae,Cortinariaceae,Hyaloriaceae,Hyphodermataceae,Hygrophoraceae,Panaceae,Clavariaceae,Botryobasidiaceae,Pleurotaceae,Lycoperdaceae,Physalacriaceae,Lachnocladiaceae,Paxillaceae,Typhulaceae,Hymenochaetales_fam_Incertae_sedis,Ischnodermataceae,Irpicaceae,Polyporales_fam_Incertae_sedis,Polyporaceae,Thelephoraceae,Nidulariaceae,Podoscyphaceae,Geastraceae,Cerrenaceae,Tapinellaceae,Hydnodontaceae,Lyophyllaceae,Marasmiaceae,Peniophoraceae,Suillaceae,Inocybaceae,Pluteaceae,Amanitaceae,Hericiaceae,Agaricaceae,Pterulaceae,Psathyrellaceae'

# mushrooms 
qiime taxa filter-table \
    --i-table Controls/decontam-table.qza \
    --i-taxonomy taxonomy.qza \
    --p-exclude $filter \
    --p-include p__ \
    --o-filtered-table Controls/non_target_filtered_table.qza

qiime feature-table filter-samples \
    --i-table dada2-3/dada-table.qza \
    --m-metadata-file /home/lfloerl/Compare-Mock-Communities/Metadata/NovaSeq6000_fun_metadata.tsv \
    --p-where "[Ctrl]='pos'" \
    --o-filtered-table Controls/pos_ctrl_table.qza

qiime feature-table filter-samples \
    --i-table dada2-3/dada-table.qza \
    --m-metadata-file /home/lfloerl/Compare-Mock-Communities/Metadata/NovaSeq6000_fun_metadata.tsv \
    --p-where "[Sample_Name] IN ('DNA24_PosCtrl_2','DNA26_PosCtrl_2','DNA30_PosCtrl_3','DNA30_PosCtrl_1','DNA29_PosCtrl_3','DNA28_PosCtrl_1','DNA26_PosCtrl_1','ctrl positive p6','ctrl positive p1','ctrl positive p3','DNA29_PosCtrl_2','DNA29_PosCtrl_1','DNA24_PosCtrl_1','DNA27_PosCtrl_20')" \
    --o-filtered-table Controls/filtered_pos_ctrl_table.qza

Saved FeatureTable[Frequency] to: Controls/non_target_filtered_table.qza
Saved FeatureTable[Frequency] to: Controls/pos_ctrl_table.qza
Saved FeatureTable[Frequency] to: Controls/filtered_pos_ctrl_table.qza


## Taxa Barplot

In [158]:
table = q2.Artifact.load('Controls/filtered_pos_ctrl_table.qza')
taxonomy =  q2.Artifact.load('taxonomy.qza')
md = Metadata.load('/home/lfloerl/Compare-Mock-Communities/Metadata/NovaSeq6000_fun_metadata.tsv')

taxa_bar_plots_viz, = taxa_actions.barplot(
    table=table,
    taxonomy=taxonomy,
    metadata=md
)
taxa_bar_plots_viz.save('Controls/taxa_barplot_all.qzv')

'Controls/taxa_barplot_all.qzv'

# 16S

In [159]:
wd = '/home/lfloerl/cloud/lfloerl/Alm-Microbiome/artifacts/16S'
%cd $wd 

/home/lfloerl/cloud/lfloerl/Alm-Microbiome/artifacts/16S


In [160]:
md = Metadata.load('/home/lfloerl/Compare-Mock-Communities/Metadata/NovaSeq6000_bac_metadata.tsv')

### Classify

In [135]:
%%bash 

qiime feature-classifier classify-sklearn \
    --i-reads  \
    --i-classifier /home/floerl/public/Data/Databases/QIIME2/SILVA/silva-138-99-515-806-nb-classifier.qza \
    --p-n-jobs 10 \
    --o-classification taxonomy.qza

qiime metadata tabulate --m-input-file taxonomy.qza --o-visualization taxonomy.qzv

Saved FeatureData[Taxonomy] to: taxonomy.qza
Saved Visualization to: taxonomy.qzv


### Decontam 

In [ ]:
%%bash

mkdir Controls

# first we need to subset the table to only contain our controls (decontam cannot handle samples missing in the metadata)
qiime feature-table filter-samples \
  --i-table dada2-2/dada-table.qza \
  --m-metadata-file /home/lfloerl/Compare-Mock-Communities/Metadata/NovaSeq6000_bac_metadata.tsv \
  --o-filtered-table Controls/ctrl_md_filtered_table.qza

qiime feature-table summarize --i-table Controls/ctrl_md_filtered_table.qza --o-visualization Controls/ctrl_md_filtered_table.qzv

# DECONTAM 
qiime quality-control decontam-identify \
    --i-table Controls/ctrl_md_filtered_table.qza \
    --m-metadata-file /home/lfloerl/Compare-Mock-Communities/Metadata/NovaSeq6000_bac_metadata.tsv \
    --p-method 'combined' \
    --p-freq-concentration-column 'Bacterial Conc' \
    --p-prev-control-column "Ctrl" \
    --p-prev-control-indicator "neg" \
    --o-decontam-scores Controls/decontam_scores.qza \

qiime quality-control decontam-score-viz \
    --i-decontam-scores Controls/decontam_scores.qza \
    --i-table Controls/ctrl_md_filtered_table.qza \
    --o-visualization Controls/decontam_histogram.qzv

# actually remove the contaminations 
qiime quality-control decontam-remove \
    --i-decontam-scores Controls/decontam_scores.qza \
    --i-table Controls/ctrl_md_filtered_table.qza \
    --o-filtered-table Controls/decontam-table.qza \

qiime feature-table summarize --i-table Controls/decontam-table.qza --o-visualization Controls/decontam-table.qzv

### Filter

In [161]:
taxonomy = q2.Artifact.load('taxonomy.qza')
decontam_table = q2.Artifact.load('Controls/decontam-table.qza')

# check how many mitochondria 
exclude_terms = 'mitochondria,chloroplast,Eukaryota,Archaea'

md_filtered_table, = filter_samples(table=decontam_table,
                                metadata=md, where="[ctrl]='pos'")

md_filtered_table2, = filter_samples(table=md_filtered_table,
                                metadata=md, where="[Sample_Name] IN ('DNA24_PosCtrl_2','DNA26_PosCtrl_2','DNA30_PosCtrl_3','DNA30_PosCtrl_1','DNA29_PosCtrl_3','DNA28_PosCtrl_1','DNA26_PosCtrl_1','ctrl positive p6','ctrl positive p1','ctrl positive p3','DNA29_PosCtrl_2','DNA29_PosCtrl_1','DNA24_PosCtrl_1','DNA27_PosCtrl_20')")
                                
taxa_filtered_table, = q2t.actions.filter_table(
                                table=md_filtered_table2, 
                                taxonomy=taxonomy,
                                exclude=exclude_terms,
                                include='p__')
                                
taxa_filtered_table.save('Controls/chloroplast_mitochondria_filtered_table.qza')

!qiime feature-table summarize --i-table Controls/chloroplast_mitochondria_filtered_table.qza --o-visualization Controls/chloroplast_mitochondria_filtered_table.qzv

Saved Visualization to: Controls/chloroplast_mitochondria_filtered_table.qzv


### Taxa Barplot

In [162]:
#table = q2.Artifact.load('Controls/pos_ctrl_table.qza')

taxa_bar_plots_viz, = taxa_actions.barplot(
    table=taxa_filtered_table,
    taxonomy=taxonomy,
    metadata=md
)
taxa_bar_plots_viz.save('Controls/taxa_barplot_all.qzv')

'Controls/taxa_barplot_all.qzv'